In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN, MeanShift
from sklearn.metrics import pairwise_distances
import seaborn as sns

from IPython.display import Markdown, display

In [2]:
df = pd.read_excel('adel.xlsx', sheet_name=1, index_col=0)

In [3]:
desired_columns = ['v' + str(x) for x in range(9, 22)]
desired_columns.append('v24')

In [4]:
df = df[desired_columns]

## DBSCAN

In [5]:
eps = 0.5

while True:
    labels = DBSCAN(eps=eps).fit_predict(df)
    if sum(labels == -1) <= 60:
        break
    
    eps += 0.1

print('eps =', eps)
print('labels =', np.unique(labels))
print('number of individual clusters =', sum(labels == -1))

table = 'Cluster|# Points\n---:|:---\n'
for i in range(len(np.unique(labels))):
    table += str(i-1) + '|'  # To account for -1
    table += str(sum(labels == i-1))
    table += '\n'

display(Markdown(table))

eps = 18.599999999999994
labels = [-1  0  1  2]
number of individual clusters = 60


Cluster|# Points
---:|:---
-1|60
0|541
1|3
2|14


## Mean Shift

In [8]:
ms = MeanShift()
preds = ms.fit_predict(df)
print(np.unique(preds))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]


In [9]:
table = 'Cluster|# Points\n---:|:---\n'
for i in range(15):
    table += str(i) + '|'
    table += str(sum(preds == i))
    table += '\n'

display(Markdown(table))

Cluster|# Points
---:|:---
0|504
1|82
2|15
3|3
4|2
5|2
6|2
7|1
8|1
9|1
10|1
11|1
12|1
13|1
14|1


In [10]:
indiv_indices = np.where(preds >= 7)[0]
preds[indiv_indices] = -1

In [11]:
print(np.unique(preds))

[-1  0  1  2  3  4  5  6]


In [12]:
table = 'Cluster|# Points\n---:|:---\n'
for i in range(-1, 7):
    table += str(i) + '|'
    table += str(sum(preds == i))
    table += '\n'

display(Markdown(table))

Cluster|# Points
---:|:---
-1|8
0|504
1|82
2|15
3|3
4|2
5|2
6|2


In [13]:
# Check the 504/541 points
dbs_indices = np.where(labels == 0)[0]
ms_indices = np.where(preds == 0)[0]

print(len(set(dbs_indices).intersection(ms_indices)))

504


In [14]:
# Double-check this result!!
print(all([x in dbs_indices for x in ms_indices]))

True


In [15]:
# Check the 541/82 points
dbs_indices = np.where(labels == 0)[0]
ms_indices = np.where(preds == 1)[0]

print(len(set(dbs_indices).intersection(ms_indices)))

37


In [16]:
# Check the 82/60 points
dbs_indices = np.where(labels == -1)[0]
ms_indices = np.where(preds == 1)[0]

print(len(set(dbs_indices).intersection(ms_indices)))

28


This means that 28 points that were individual clusters according to DBSCAN were a part of a cluster according to Mean Shift.

In [17]:
# Check the 82/14 points
dbs_indices = np.where(labels == 2)[0]
ms_indices = np.where(preds == 1)[0]

print(len(set(dbs_indices).intersection(ms_indices)))

14


This means that apart from the 28 that were marked by DBSCAN as individual clusters, all 14 points that it had marked as one cluster were also part of this cluster of 82 points.

In [18]:
# Check the 82/3 points
dbs_indices = np.where(labels == 1)[0]
ms_indices = np.where(preds == 1)[0]

print(len(set(dbs_indices).intersection(ms_indices)))

3


To summarize:  

* DBSCAN identified 3 clusters, 0-2, and identified 60 points as not belonging to any cluster. This is because of the threshold that we set.
* Mean-Shift identified 7 clusters, 0-6, and identified 8 points as not belonging to any cluster.
* All of the MS cluster 0 points were a part of DBSCAN cluster 0.
* Out of 82, 37 of MS cluster 1 points were also a part of DBSCAN cluster 0  

We'll put the full summary in a table.

In [19]:
table = 'Index|DBSCAN cluster#|MS cluster#|DBSCAN cluster count|MS cluster count|Common\n---|:---:|:---:|:---:|:---:|---\n'
index = 1

for i in range(-1, 3):
    dbs_count = len(np.where(labels == i)[0])
    
    for j in range(-1, 7):
        ms_count = len(np.where(preds == j)[0])
        
        dbs_indices = np.where(labels == i)[0]
        ms_indices = np.where(preds == j)[0]
        
        common = len(set(dbs_indices).intersection(ms_indices))
        
        if common > 0:
            table += '{0}|{1}|{2}|{3}|{4}|{5}\n'.format(index, i, j, dbs_count, ms_count, common)
            index += 1

display(Markdown(table))

Index|DBSCAN cluster#|MS cluster#|DBSCAN cluster count|MS cluster count|Common
---|:---:|:---:|:---:|:---:|---
1|-1|-1|60|8|8
2|-1|1|60|82|28
3|-1|2|60|15|15
4|-1|3|60|3|3
5|-1|4|60|2|2
6|-1|5|60|2|2
7|-1|6|60|2|2
8|0|0|541|504|504
9|0|1|541|82|37
10|1|1|3|82|3
11|2|1|14|82|14


So an English summary is as follows.  

* **Individual clusters discussion:** 
  * Because of our threshold, DBSCAN marked 60 points as individual clusters. Mean-Shift marked only 8 such points **(row 1)**
  * However, Mean-Shift marked 4 other clusters that had 3 or less points. All points marked by Mean-Shift as having 3 or less points were a part of DBSCAN's individual clusters **(rows 4-7)**. 
  * More surprisingly, a cluster marked by Mean-Shift as having 15 points was marked by DBSCAN as 15 individual clusters **(row 3)**. So far, we've accounted for 32/60 of DBSCAN's individual clusters. 
  * The rest of the 28 come from Mean-Shift's second largest cluster of 82 points **(row 2)**. This analysis may suggest that we should lower the threshold for DBSCAN, or *increase* `eps`.
* **Largest clusters discussion:**
  * Mean-Shift's largest cluster had 504 points. This cluster was a proper subset of DBSCAN's largest subset of 541 points **(row 8)**.
  * A large part of Mean-Shift's second-largest cluster, 37/82 points, was also absorbed by DBSCAN's largest cluster **(row 9)**. 
  * Perhaps reducing `eps` might make DBSCAN's clusters smaller, and make both the clustering algorithms' outputs more similar. While this seems to contradict an earlier point, these are in fact two separate observations. Increasing `eps` would mean DBSCAN would find less clusters as single points. Decreasing `eps` would certainly increase that, which we do not want, but would shift some points in the larger clusters around, which may be desirable.

## Finding "prolific" authors using DBSCAN clusters

### Cluster 0 and cluster 2 (biggest and second-biggest)

In [20]:
import scipy.stats as st

In [21]:
dbs0 = df.iloc[np.where(labels == 0)[0],:]

In [22]:
dbs0.mean()

v9      5.127542
v10     9.171904
v11     3.994455
v12     1.908170
v13     3.430684
v14    22.172606
v15     3.789593
v16     8.525139
v17     7.510222
v18     2.900924
v19     4.071405
v20     0.216147
v21     0.249316
v24     0.240296
dtype: float64

In [23]:
dbs2 = df.iloc[np.where(labels == 2)[0],:]

In [24]:
dbs2.mean()

v9      16.857143
v10     32.642857
v11     12.642857
v12      4.439286
v13      9.785714
v14    157.159286
v15     12.535000
v16     44.418571
v17     28.739286
v18      7.440000
v19     18.269286
v20      0.352442
v21      0.481429
v24      4.214286
dtype: float64

In [25]:
pvalues = st.ttest_ind(dbs0, dbs2, equal_var=False)[1]

In [26]:
all(pvalues <= 0.05)

True

In [27]:
dbs_ind = df.iloc[np.where(labels == -1)[0],:]
dbs_ind.mean()

v9      24.383333
v10     46.716667
v11     15.566667
v12      8.383833
v13     15.433333
v14    296.113833
v15     16.244833
v16    108.879333
v17     37.859000
v18     13.961500
v19     43.061000
v20      0.615387
v21      0.597667
v24      5.650000
dtype: float64

### Cluster 0 and cluster 1

In [28]:
dbs1 = df.iloc[np.where(labels == 1)[0],:]

In [29]:
pvalues = st.ttest_ind(dbs0, dbs1, equal_var=False)[1]
print(all(pvalues <= 0.05))

False


In [30]:
print(sum(pvalues <= 0.05) / len(pvalues))

0.7857142857142857


10/14 p-values are statistically significant.

### All pairs

In [31]:
table = 'Cluster 1|Cluster 2|95% significant|90% significant\n---:|:---:|:---:|:---\n'

for i in range(-1, 3):
    dbs_i =  df.iloc[np.where(labels == i)[0],:]
    
    for j in range(i+1, 3):
        dbs_j = df.iloc[np.where(labels == j)[0],:]
        
        pvalues_95 = st.ttest_ind(dbs_i, dbs_j, equal_var=False)[1]
        per_significant_95 = sum(pvalues_95 <= 0.05) / len(pvalues)
        
        pvalues_90 = st.ttest_ind(dbs_i, dbs_j, equal_var=False)[1]
        per_significant_90 = sum(pvalues_90 <= 0.1) / len(pvalues)
        table += '{0}|{1}|{2}|{3}\n'.format(i, j, per_significant_95, per_significant_90)

display(Markdown(table))

Cluster 1|Cluster 2|95% significant|90% significant
---:|:---:|:---:|:---
-1|0|1.0|1.0
-1|1|0.8571428571428571|0.8571428571428571
-1|2|0.9285714285714286|1.0
0|1|0.7857142857142857|0.8571428571428571
0|2|1.0|1.0
1|2|0.6428571428571429|0.7857142857142857


So the biggest cluster and the individual clusters have means that are statistically significant at 95% confidence level. The biggest and second biggest clusters (0 and 2) also have means that are statistically significant at 95% confidence level. For the second biggest and the individual clusters, 13/14 p-values are statistically significant at 95% confidence level, and all of these are at 90% confidence level.

In [32]:
from sklearn.metrics import silhouette_score

In [33]:
silhouette_score(df, labels)

0.5713389151552444

In [34]:
pvalues_95 = st.ttest_ind(dbs_ind, dbs2, equal_var=False)[1]
np.where(pvalues_95 > 0.05)[0]

array([13])

## Clustering on $S_1$

In [37]:
desired_columns = ['v6','v9','v11','v13','v14','v17','v19','v24','v25']
df = pd.read_excel('adel.xlsx', sheet_name=1, index_col=0)
df = df[desired_columns]

In [38]:
eps = 0.5

while True:
    labels = DBSCAN(eps=eps).fit_predict(df)
    if sum(labels == -1) <= 60:
        break
    
    eps += 0.1

print('eps =', eps)
print('labels =', np.unique(labels))
print('number of individual clusters =', sum(labels == -1))

table = 'Cluster|# Points\n---:|:---\n'

for i in range(len(np.unique(labels))):
    table += str(i-1) + '|'  # To account for -1
    table += str(sum(labels == i-1))
    table += '\n'

display(Markdown(table))

eps = 45.00000000000037
labels = [-1  0  1  2]
number of individual clusters = 60


Cluster|# Points
---:|:---
-1|60
0|532
1|20
2|6


In [39]:
ms = MeanShift()
preds = ms.fit_predict(df)
print(np.unique(preds))

table = 'Cluster|# Points\n---:|:---\n'
for i in range(len(np.unique(preds))):
    table += str(i) + '|'
    table += str(sum(preds == i))
    table += '\n'

display(Markdown(table))

[ 0  1  2  3  4  5  6  7  8  9 10 11]


Cluster|# Points
---:|:---
0|509
1|72
2|19
3|4
4|4
5|3
6|1
7|1
8|1
9|2
10|1
11|1


In [44]:
indiv_indices = np.where(preds >= 6)[0]
rm_indices = np.where(preds == 9)[0]

indiv_indices = list(set(indiv_indices).difference(rm_indices))

In [45]:
indiv_indices

[0, 3, 4, 55, 217]

In [46]:
preds[indiv_indices] = -1

In [47]:
print(np.unique(preds))

[-1  0  1  2  3  4  5  9]


In [48]:
indices = np.where(preds == 9)[0]
preds[indices] = 6

In [49]:
table = 'Cluster|# Points\n---:|:---\n'
for i in range(-1, 7):
    table += str(i) + '|'
    table += str(sum(preds == i))
    table += '\n'

display(Markdown(table))

Cluster|# Points
---:|:---
-1|5
0|509
1|72
2|19
3|4
4|4
5|3
6|2


In [50]:
table = 'Index|DBSCAN cluster#|MS cluster#|DBSCAN cluster count|MS cluster count|Common\n---|:---:|:---:|:---:|:---:|---\n'
index = 1

for i in range(-1, 3):
    dbs_count = len(np.where(labels == i)[0])
    
    for j in range(-1, 7):
        ms_count = len(np.where(preds == j)[0])
        
        dbs_indices = np.where(labels == i)[0]
        ms_indices = np.where(preds == j)[0]
        
        common = len(set(dbs_indices).intersection(ms_indices))
        
        if common > 0:
            table += '{0}|{1}|{2}|{3}|{4}|{5}\n'.format(index, i, j, dbs_count, ms_count, common)
            index += 1

display(Markdown(table))

Index|DBSCAN cluster#|MS cluster#|DBSCAN cluster count|MS cluster count|Common
---|:---:|:---:|:---:|:---:|---
1|-1|-1|60|5|5
2|-1|0|60|509|2
3|-1|1|60|72|21
4|-1|2|60|19|19
5|-1|3|60|4|4
6|-1|4|60|4|4
7|-1|5|60|3|3
8|-1|6|60|2|2
9|0|0|532|509|507
10|0|1|532|72|25
11|1|1|20|72|20
12|2|1|6|72|6


An English summary is as follows:  
* The largest clusters for both the algorithms are very similar (**row 9**). The two points in the MS cluster that were not in DBSCAN's largest cluster were in its second largest cluster (**row 2**).
* All the clusters of the mean-shift algorithm that were of size 4 or less were marked as individual clusters by DBSCAN (**rows 5-8 and row 1**). More surprisingly, the cluster of 19 points identified by mean-shift was marked as 19 individual clusters by DBSCAN (**row 4**). A considerable part of mean-shift's second-largest cluster was also identified as individual points by DBSCAN (**row 3**).
* All of DBSCAN's other clusters were absorbed by mean-shift's second largest cluster (**rows 11-12**).

In [51]:
silhouette_score(df, labels)

0.6351765089361389